In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import xgboost

In [2]:
DATA_PATH = "."

In [3]:
full_train = utils.load_train_hdf(DATA_PATH)

In [4]:
closest_hits_features = full_train.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████| 5445705/5445705 [2:58:18<00:00, 509.00it/s]  


In [5]:
train_concat = pd.concat(
    [full_train.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

In [6]:
model = xgboost.XGBClassifier(n_jobs=-1, n_estimators=200, max_depth=5)


In [7]:
model.fit(train_concat.values, full_train.label, sample_weight=full_train.weight)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [8]:
import pickle

pickle.dump(model, open("pima.pickle.dat", "wb"))

In [9]:
full_test = utils.load_public_test_hdf()

In [10]:
closest_hits_features = full_test.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████| 726095/726095 [14:34<00:00, 829.92it/s]


In [11]:
test_concat = pd.concat(
    [full_test.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

In [12]:
predictions = model.predict_proba(test_concat.values)


In [13]:
pd.DataFrame(data={"prediction": predictions[:, 1]}, index=test_concat.index).to_csv(
    "sssssssssssss_submission.csv", index_label=utils.ID_COLUMN)

In [15]:
full_test_priv = utils.load_private_test_hdf()

In [16]:
closest_hits_features = full_test_priv.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████| 1452188/1452188 [29:37<00:00, 816.76it/s]


In [17]:
test_concat_priv = pd.concat(
    [full_test_priv.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

In [18]:
predictions = model.predict_proba(test_concat_priv.values)

In [20]:
pd.DataFrame(data={"prediction": predictions[:, 1]}, index=test_concat_priv.index).to_csv(
    "pechat.csv", index_label=utils.ID_COLUMN)